In [1]:
import numpy as np
import random

In [5]:
class Agent:
    def __init__(self, action_space, state_space):
        self.action_space = action_space  # Lista de ações possíveis
        self.state_space = state_space  # Lista de estados possíveis
        self.q_table = np.zeros((len(state_space), len(action_space)))  # Q-Learning (Podemos mudar pra D-Learning visto em aula 16/09)
        self.learning_rate = 0.1
        self.discount_factor = 0.9
        self.epsilon = 0.1  # Exploração

    def choose_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(self.action_space)  # Explorar
        else:
            return np.argmax(self.q_table[state])  # Exploitar (melhor ação)

    def update_q_table(self, state, action, reward, next_state):
        best_next_action = np.max(self.q_table[next_state])
        self.q_table[state, action] = self.q_table[state, action] + self.learning_rate * (reward + self.discount_factor * best_next_action - self.q_table[state, action])

# Agente codificador (Exemplos)
action_space = ["limpar_dados", "analisar_dados", "visualizar_resultados"]
state_space = ["dados_brutos", "dados_processados", "resultados_gerados"]

codificador = Agent(action_space, state_space)
# Agente revisor (Exemplos)
action_space_revisor = ["executar_codigo", "propor_refatoracao", "analisar_estaticamente", "aprovar_codigo"]
state_space_revisor = ["codigo_gerado", "codigo_com_erro", "codigo_pronto"]

revisor = Agent(action_space_revisor, state_space_revisor)

In [3]:
class Ambiente:
    def __init__(self, codificador, revisor, task):
        self.codificador = codificador
        self.revisor = revisor
        self.task = task  # Definir uma tarefa de análise de dados (por exemplo, limpeza de dados)
    
    def executar_ciclo(self):
        state = self.task.initial_state()
        
        # Ciclo de codificação
        action = self.codificador.choose_action(state)
        next_state, reward_codificador = self.task.executar_acao_codificador(action)
        self.codificador.update_q_table(state, action, reward_codificador, next_state)
        
        # Ciclo de revisão
        action_revisor = self.revisor.choose_action(next_state)
        final_state, reward_revisor = self.task.executar_acao_revisor(action_revisor)
        self.revisor.update_q_table(next_state, action_revisor, reward_revisor, final_state)
        
        return final_state


In [6]:
class Task:
    def __init__(self, dataset):
        self.dataset = dataset  # Dataset que vamos usar
        self.state = "dados_brutos" # Estado inicial
    
    def initial_state(self):
        return self.state
    
    # Exemplo de ciclo com penalidade e recompensa (Vamos deixar mais complexo inserindo penalidade por caminhos mais longos no futuro)
    def executar_acao_codificador(self, action):
        if action == "limpar_dados":
            # Lógica de limpeza dos dados
            self.state = "dados_processados"
            reward = 10  # Exemplo de recompensa
        elif action == "analisar_dados":
            self.state = "resultados_gerados"
            reward = 20
        else:
            reward = -10  # Penalidade por erro
        return self.state, reward
    
    def executar_acao_revisor(self, action):
        if action == "executar_codigo":
            # Simular execução do código gerado
            reward = 5
        elif action == "propor_refatoracao":
            reward = 15
        else:
            reward = -5  # Penalidade
        return self.state, reward
